In [1]:
import logging
import time
import threading
from contextlib import contextmanager
from typing import Dict, Any, Optional, Callable, List
from dataclasses import dataclass
from enum import Enum
import functools

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("ResourceManager")

In [2]:
class ResourceType(Enum):
    DATABASE = "database"
    API = "api"
    FILE = "file"
    NETWORK = "network"

class ResourceState(Enum):
    INITIALIZED = "initialized"
    CONNECTED = "connected"
    DISCONNECTED = "disconnected"
    ERROR = "error"

In [3]:
@dataclass
class ResourceMetrics:
    connection_time: float = 0.0
    query_count: int = 0
    error_count: int = 0
    total_bytes_transferred: int = 0
    last_activity: float = 0.0

@dataclass
class ResourceConfig:
    name: str
    resource_type: ResourceType
    connection_string: str
    timeout: float = 30.0
    max_retries: int = 3
    cleanup_timeout: float = 10.0

class ResourceException(Exception):
    """Custom exception for resource management errors"""
    pass

In [4]:
class DatabaseConnection:
    """Simulated database connection with proper method signatures"""
    def __init__(self, config: ResourceConfig):
        self.config = config
        self.is_connected = True
    
    def execute(self, query: str) -> str:
        logger.debug(f"Executing database query: {query}")
        return f"Result for: {query}"
    
    def close(self):
        self.is_connected = False
        logger.debug("Database connection closed")

In [5]:
class APIConnection:
    """Simulated API connection with proper method signatures"""
    def __init__(self, config: ResourceConfig):
        self.config = config
        self.is_connected = True
        self.base_url = config.connection_string
    
    def get(self, endpoint: str) -> dict:
        logger.debug(f"API GET request to: {self.base_url}/{endpoint}")
        return {"status": "success", "data": f"response from {endpoint}"}
    
    def post(self, endpoint: str, data: dict) -> dict:
        logger.debug(f"API POST request to: {self.base_url}/{endpoint} with data: {data}")
        return {"status": "success", "id": 123}
    
    def disconnect(self):
        self.is_connected = False
        logger.debug("API connection disconnected")

In [6]:
class FileHandle:
    """Simulated file handle with proper method signatures"""
    def __init__(self, config: ResourceConfig):
        self.config = config
        self.filename = config.connection_string
        self.is_open = True
    
    def read(self) -> str:
        logger.debug(f"Reading from file: {self.filename}")
        return f"Content from {self.filename}"
    
    def write(self, data: str) -> bool:
        logger.debug(f"Writing to file: {self.filename} - Data: {data}")
        return True
    
    def close(self):
        self.is_open = False
        logger.debug("File handle closed")

In [7]:
class NetworkConnection:
    """Simulated network connection with proper method signatures"""
    def __init__(self, config: ResourceConfig):
        self.config = config
        self.is_connected = True
        self.address = config.connection_string
    
    def send(self, data: str) -> bool:
        logger.debug(f"Sending data via network: {data}")
        return True
    
    def receive(self) -> str:
        logger.debug("Receiving data via network")
        return "Received network data"
    
    def disconnect(self):
        self.is_connected = False
        logger.debug("Network connection disconnected")

In [8]:
class ResourceManager:
    """
    A robust context manager for managing multiple external resources
    with proper cleanup, logging, and performance metrics.
    """
    
    _thread_local = threading.local()
    
    def __init__(self, configs: List[ResourceConfig], enable_metrics: bool = True):
        self.configs = {config.name: config for config in configs}
        self.resources: Dict[str, Any] = {}
        self.metrics: Dict[str, ResourceMetrics] = {}
        self.state: Dict[str, ResourceState] = {}
        self.enable_metrics = enable_metrics
        self._lock = threading.RLock()
        self._nested_level = 0
        
    @classmethod
    def _get_context_stack(cls) -> List['ResourceManager']:
        """Get the current context stack for nested managers"""
        if not hasattr(cls._thread_local, 'context_stack'):
            cls._thread_local.context_stack = []
        return cls._thread_local.context_stack
    
    def __enter__(self):
        """Enter the context manager"""
        context_stack = self._get_context_stack()
        context_stack.append(self)
        self._nested_level = len(context_stack) - 1
        
        logger.info(f"Entering ResourceManager context (level {self._nested_level})")
        
        try:
            self._connect_all_resources()
            return self
        except Exception as e:
            self._safe_cleanup()
            raise ResourceException(f"Failed to enter resource context: {e}") from e
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Exit the context manager with proper cleanup"""
        context_stack = self._get_context_stack()
        if context_stack and context_stack[-1] is self:
            context_stack.pop()
        
        logger.info(f"Exiting ResourceManager context (level {self._nested_level})")
        
        try:
            if exc_type is not None:
                logger.error(f"Context exited with exception: {exc_type.__name__}: {exc_val}")
                self._safe_cleanup()
            else:
                self._disconnect_all_resources()
        except Exception as cleanup_error:
            logger.error(f"Error during resource cleanup: {cleanup_error}")
            if exc_type is None:  # Don't mask original exception
                raise
        
        self._nested_level = 0
        return False  # Don't suppress exceptions
    
    def _connect_all_resources(self):
        """Connect to all configured resources"""
        with self._lock:
            for name, config in self.configs.items():
                if name not in self.resources:
                    self._connect_resource(name, config)
    
    def _connect_resource(self, name: str, config: ResourceConfig):
        """Connect to a specific resource"""
        start_time = time.time()
        
        try:
            logger.info(f"Connecting to {config.resource_type.value}: {name}")
            
            # Create appropriate resource based on type
            if config.resource_type == ResourceType.DATABASE:
                resource = DatabaseConnection(config)
            elif config.resource_type == ResourceType.API:
                resource = APIConnection(config)
            elif config.resource_type == ResourceType.FILE:
                resource = FileHandle(config)
            elif config.resource_type == ResourceType.NETWORK:
                resource = NetworkConnection(config)
            else:
                raise ResourceException(f"Unsupported resource type: {config.resource_type}")
            
            self.resources[name] = resource
            self.state[name] = ResourceState.CONNECTED
            
            if self.enable_metrics:
                self.metrics[name] = ResourceMetrics(
                    connection_time=time.time() - start_time,
                    last_activity=time.time()
                )
            
            logger.info(f"Successfully connected to {name} in {time.time() - start_time:.3f}s")
            
        except Exception as e:
            self.state[name] = ResourceState.ERROR
            logger.error(f"Failed to connect to {name}: {e}")
            raise ResourceException(f"Connection failed for {name}: {e}") from e
    
    def _disconnect_all_resources(self):
        """Disconnect all resources"""
        with self._lock:
            for name in list(self.resources.keys()):
                self._disconnect_resource(name)
    
    def _disconnect_resource(self, name: str):
        """Disconnect a specific resource"""
        if name not in self.resources:
            return
        
        config = self.configs[name]
        start_time = time.time()
        
        try:
            logger.info(f"Disconnecting from {name}")
            resource = self.resources[name]
            
            # Proper resource cleanup
            if hasattr(resource, 'close'):
                resource.close()
            elif hasattr(resource, 'disconnect'):
                resource.disconnect()
            
            del self.resources[name]
            self.state[name] = ResourceState.DISCONNECTED
            
            logger.info(f"Successfully disconnected from {name} in {time.time() - start_time:.3f}s")
            
        except Exception as e:
            logger.error(f"Error disconnecting from {name}: {e}")
            self.state[name] = ResourceState.ERROR
    
    def _safe_cleanup(self):
        """Safe cleanup that handles exceptions"""
        with self._lock:
            for name in list(self.resources.keys()):
                try:
                    self._disconnect_resource(name)
                except Exception as e:
                    logger.error(f"Error during safe cleanup of {name}: {e}")
    
    # Public API methods
    def get_resource(self, name: str) -> Any:
        """Get a resource by name"""
        if name not in self.resources:
            raise ResourceException(f"Resource not found: {name}")
        
        if self.enable_metrics:
            self.metrics[name].last_activity = time.time()
        
        return self.resources[name]
    
    def execute_query(self, resource_name: str, query: str) -> Any:
        """Execute a query on a resource with metrics tracking"""
        if self.enable_metrics:
            self.metrics[resource_name].query_count += 1
        
        resource = self.get_resource(resource_name)
        
        if hasattr(resource, 'execute'):
            return resource.execute(query)
        else:
            raise ResourceException(f"Resource {resource_name} doesn't support execute")
    
    def api_get(self, resource_name: str, endpoint: str) -> dict:
        """Make a GET request to an API resource"""
        if self.enable_metrics:
            self.metrics[resource_name].query_count += 1
        
        resource = self.get_resource(resource_name)
        
        if hasattr(resource, 'get'):
            return resource.get(endpoint)
        else:
            raise ResourceException(f"Resource {resource_name} doesn't support GET requests")
    
    def api_post(self, resource_name: str, endpoint: str, data: dict) -> dict:
        """Make a POST request to an API resource"""
        if self.enable_metrics:
            self.metrics[resource_name].query_count += 1
        
        resource = self.get_resource(resource_name)
        
        if hasattr(resource, 'post'):
            return resource.post(endpoint, data)
        else:
            raise ResourceException(f"Resource {resource_name} doesn't support POST requests")
    
    def get_metrics(self, resource_name: Optional[str] = None) -> Dict[str, Any]:
        """Get performance metrics for resources"""
        if not self.enable_metrics:
            return {}
        
        if resource_name:
            return {resource_name: self.metrics.get(resource_name)}
        else:
            return self.metrics.copy()
    
    def get_state(self, resource_name: Optional[str] = None) -> Dict[str, ResourceState]:
        """Get the state of resources"""
        if resource_name:
            return {resource_name: self.state.get(resource_name, ResourceState.DISCONNECTED)}
        else:
            return self.state.copy()
    
    def is_connected(self, resource_name: str) -> bool:
        """Check if a specific resource is connected"""
        return self.state.get(resource_name) == ResourceState.CONNECTED

In [9]:
# Decorator for automatic resource management
def with_resources(configs: List[ResourceConfig]):
    """Decorator to automatically manage resources for a function"""
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            with ResourceManager(configs) as manager:
                # Inject resource manager as first argument if function expects it
                import inspect
                sig = inspect.signature(func)
                if 'resource_manager' in sig.parameters:
                    kwargs['resource_manager'] = manager
                return func(*args, **kwargs)
        return wrapper
    return decorator

In [10]:
def main():
    """Demonstrate the resource manager in action"""
    
    # Configure resources
    configs = [
        ResourceConfig("main_db", ResourceType.DATABASE, "postgresql://localhost/main"),
        ResourceConfig("api_service", ResourceType.API, "https://api.example.com"),
        ResourceConfig("log_file", ResourceType.FILE, "/var/log/app.log"),
        ResourceConfig("cache_server", ResourceType.NETWORK, "redis://localhost:6379"),
    ]
    
    print("=== Basic Resource Manager Usage ===")
    
    # Basic usage
    with ResourceManager(configs) as manager:
        # Get and use resources
        db = manager.get_resource("main_db")
        api = manager.get_resource("api_service")
        
        # Execute queries - this should work now
        result = manager.execute_query("main_db", "SELECT * FROM users")
        print(f"Query result: {result}")
        
        # API calls
        api_response = manager.api_get("api_service", "/users")
        print(f"API response: {api_response}")
        
        # Check metrics
        metrics = manager.get_metrics()
        print(f"Metrics: {metrics}")
        
        # Check states
        states = manager.get_state()
        print(f"States: {states}")
    
    print("\n=== Nested Context Managers ===")
    
    # Nested usage
    with ResourceManager([configs[0]]) as outer_manager:
        print(f"Outer context - DB state: {outer_manager.get_state('main_db')}")
        print(f"Outer context - DB connected: {outer_manager.is_connected('main_db')}")
        
        with ResourceManager([configs[1]]) as inner_manager:
            print(f"Inner context - API state: {inner_manager.get_state('api_service')}")
            print(f"Inner context - API connected: {inner_manager.is_connected('api_service')}")
    
    print("\n=== Using Decorator ===")
    
    # Using decorator
    @with_resources([configs[0], configs[1]])
    def process_data(resource_manager=None):
        db = resource_manager.get_resource("main_db")
        api = resource_manager.get_resource("api_service")
        
        # Use the resources
        db_result = resource_manager.execute_query("main_db", "SELECT * FROM products")
        api_result = resource_manager.api_get("api_service", "/products")
        
        print("Processing data with decorated function")
        return f"DB: {db_result}, API: {api_result}"
    
    result = process_data()
    print(f"Decorator result: {result}")
    
    print("\n=== Error Handling ===")
    
    # Error handling demonstration
    try:
        with ResourceManager(configs) as manager:
            print("Operations in progress...")
            # Try to use a non-existent resource
            try:
                manager.get_resource("non_existent")
            except ResourceException as e:
                print(f"Expected error caught: {e}")
            
            # Simulate an error in the main context
            raise ValueError("Something went wrong in the main context!")
    except ValueError as e:
        print(f"Caught main context exception: {e}")
        # Resources should be properly cleaned up despite the exception

    print("\n=== Resource-Specific Operations ===")
    
    # Demonstrate different resource types
    with ResourceManager(configs) as manager:
        # File operations
        file_handle = manager.get_resource("log_file")
        content = file_handle.read()
        print(f"File content: {content}")
        
        # Network operations
        network = manager.get_resource("cache_server")
        success = network.send("cache_data")
        print(f"Network send successful: {success}")
        
        # Final metrics
        print(f"Final metrics: {manager.get_metrics()}")

In [11]:
if __name__ == "__main__":
    main()

2025-10-06 12:00:44,716 - ResourceManager - INFO - Entering ResourceManager context (level 0)
2025-10-06 12:00:44,720 - ResourceManager - INFO - Connecting to database: main_db
2025-10-06 12:00:44,722 - ResourceManager - INFO - Successfully connected to main_db in 0.001s
2025-10-06 12:00:44,723 - ResourceManager - INFO - Connecting to api: api_service
2025-10-06 12:00:44,725 - ResourceManager - INFO - Successfully connected to api_service in 0.002s
2025-10-06 12:00:44,728 - ResourceManager - INFO - Connecting to file: log_file
2025-10-06 12:00:44,729 - ResourceManager - INFO - Successfully connected to log_file in 0.002s
2025-10-06 12:00:44,731 - ResourceManager - INFO - Connecting to network: cache_server
2025-10-06 12:00:44,733 - ResourceManager - INFO - Successfully connected to cache_server in 0.002s
2025-10-06 12:00:44,735 - ResourceManager - INFO - Exiting ResourceManager context (level 0)
2025-10-06 12:00:44,737 - ResourceManager - INFO - Disconnecting from main_db
2025-10-06 12

=== Basic Resource Manager Usage ===
Query result: Result for: SELECT * FROM users
API response: {'status': 'success', 'data': 'response from /users'}
Metrics: {'main_db': ResourceMetrics(connection_time=0.0014421939849853516, query_count=1, error_count=0, total_bytes_transferred=0, last_activity=1759748444.7345946), 'api_service': ResourceMetrics(connection_time=0.0022995471954345703, query_count=1, error_count=0, total_bytes_transferred=0, last_activity=1759748444.7346714), 'log_file': ResourceMetrics(connection_time=0.0019626617431640625, query_count=0, error_count=0, total_bytes_transferred=0, last_activity=1759748444.7297132), 'cache_server': ResourceMetrics(connection_time=0.0016677379608154297, query_count=0, error_count=0, total_bytes_transferred=0, last_activity=1759748444.733171)}
States: {'main_db': <ResourceState.CONNECTED: 'connected'>, 'api_service': <ResourceState.CONNECTED: 'connected'>, 'log_file': <ResourceState.CONNECTED: 'connected'>, 'cache_server': <ResourceState.